In [1]:
#import libraries
import pandas as pd
print('pandas: %s' % pd.__version__)
from math import sqrt
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 150)
from sklearn.preprocessing import StandardScaler

import numpy as np
print('geopandas: %s' % np.__version__)
from sklearn.metrics import brier_score_loss
import os
# Tensorflow / Keras
import tensorflow as tf # used to access argmax function
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

import decimal
from decimal import Decimal

import keras 
import tensorflow as tf
from tensorflow.keras.layers import Dropout,BatchNormalization,Conv2D,MaxPooling2D,Dense,Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from keras import callbacks
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

from pathlib import Path  

from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.preprocessing import OrdinalEncoder # for encoding labels
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

from tensorflow.keras.utils import plot_model

from sklearn.utils.class_weight import compute_sample_weight

from sklearn.calibration import CalibrationDisplay

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn import metrics


pandas: 1.4.4
geopandas: 1.22.3
Tensorflow/Keras: 2.9.0
pandas: 1.4.4
numpy: 1.22.3


In [10]:
def tom_generate(df):
    new_traning_sets = df
    new_traning_sets['Real_tom_target'] = new_traning_sets['new_target']
    rows, cols =  len(df.latitude.unique()), len(df.longitude.unique())
    size_rc = rows * cols
    for i in range(0,int((new_traning_sets.shape[0]/size_rc)-1)):
        j=i+1
        each_row2 = new_traning_sets.iloc[size_rc*j:size_rc*(j+1),:]
        new_traning_sets.iloc[size_rc*i:size_rc*(i+1),13:14] = each_row2['new_target'].values
    print("======== Done ========")
    return new_traning_sets

In [11]:
def extract_images_new(df, n_filters, verbose=False):
    times = df.time.unique()
    number_of_img, rows, cols = len(times), len(df.latitude.unique()), len(df.longitude.unique())
    images = np.zeros( (number_of_img, rows, cols, n_filters) )
    
    df = df.set_index(['time','latitude','longitude'], drop=True)
    df.sort_index(level=['time','latitude', 'longitude'], ascending=[1,0,1], inplace=True)
    k=0
    
    for day in range(0,number_of_img):
        
        images[k,:,:,:] = df.loc(axis=0)[times[day]].values.reshape(rows,cols,n_filters)
        if (k%100==0) & (verbose==True): print(k)
        k += 1
    return images

In [2]:
test = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Earth/9vars/test_earth_9var.csv')
#test = test.loc[test.time>='2016-04-01']
#train = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Windows/10vars/training_windows_10var.csv')
#val = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Windows/10vars/valid_windows_10var.csv')
#print("loading datas")
test

,Unnamed: 0,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,0,2016-01-01,90.0,-180.0,-0.000055,63.153320,-9.171402,-1.816597,-13.858635,5.202774,-166.95334,271.45947,0.0,0.0,0.0
1,1,2016-01-01,90.0,-177.5,-0.000055,62.559265,-9.667664,-1.664066,-13.267357,5.269211,-166.92370,271.45947,0.0,0.0,0.0
2,2,2016-01-01,90.0,-175.0,-0.000055,61.945680,-10.136597,-1.500095,-12.661011,5.333832,-166.73407,271.45947,0.0,0.0,0.0
3,3,2016-01-01,90.0,-172.5,-0.000055,61.326180,-10.576843,-1.324284,-12.041168,5.394989,-166.61555,271.45947,0.0,0.0,0.0
4,4,2016-01-01,90.0,-170.0,-0.000056,60.712692,-10.988846,-1.136375,-11.409485,5.451176,-166.57408,271.45947,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26879179,3836875,2022-12-31,-90.0,167.5,0.000016,97.887580,-1.422317,2.364288,-3.507492,0.720871,-182.42667,0.00000,0.0,0.0,0.0
26879180,3836876,2022-12-31,-90.0,170.0,0.000016,97.870000,-1.215286,2.370148,-3.650070,0.809738,-182.44444,0.00000,0.0,0.0,0.0
26879181,3836877,2022-12-31,-90.0,172.5,0.000017,97.856330,-1.002396,2.372101,-3.778976,0.898605,-182.46222,0.00000,0.0,0.0,0.0
26879182,3836878,2022-12-31,-90.0,175.0,0.000017,97.844610,-0.783646,2.370148,-3.892258,0.987473,-182.58667,0.00000,0.0,0.0,0.0


In [3]:
ds = test
ds = ds.drop(columns=['new_target'])
ds = ds.drop(columns=['Unnamed: 0'])
ds.rename(columns={"Real_tom_target": "new_target"}, inplace=True)
ds

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target
0,2016-01-01,90.0,-180.0,-0.000055,63.153320,-9.171402,-1.816597,-13.858635,5.202774,-166.95334,271.45947,0.0,0.0
1,2016-01-01,90.0,-177.5,-0.000055,62.559265,-9.667664,-1.664066,-13.267357,5.269211,-166.92370,271.45947,0.0,0.0
2,2016-01-01,90.0,-175.0,-0.000055,61.945680,-10.136597,-1.500095,-12.661011,5.333832,-166.73407,271.45947,0.0,0.0
3,2016-01-01,90.0,-172.5,-0.000055,61.326180,-10.576843,-1.324284,-12.041168,5.394989,-166.61555,271.45947,0.0,0.0
4,2016-01-01,90.0,-170.0,-0.000056,60.712692,-10.988846,-1.136375,-11.409485,5.451176,-166.57408,271.45947,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26879179,2022-12-31,-90.0,167.5,0.000016,97.887580,-1.422317,2.364288,-3.507492,0.720871,-182.42667,0.00000,0.0,0.0
26879180,2022-12-31,-90.0,170.0,0.000016,97.870000,-1.215286,2.370148,-3.650070,0.809738,-182.44444,0.00000,0.0,0.0
26879181,2022-12-31,-90.0,172.5,0.000017,97.856330,-1.002396,2.372101,-3.778976,0.898605,-182.46222,0.00000,0.0,0.0
26879182,2022-12-31,-90.0,175.0,0.000017,97.844610,-0.783646,2.370148,-3.892258,0.987473,-182.58667,0.00000,0.0,0.0


In [7]:
df11 = tom_generate(ds)
df11

======== Done ========


,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,2016-01-01,90.0,-180.0,-0.000055,63.153320,-9.171402,-1.816597,-13.858635,5.202774,-166.95334,271.45947,0.0,0.0,0.0
1,2016-01-01,90.0,-177.5,-0.000055,62.559265,-9.667664,-1.664066,-13.267357,5.269211,-166.92370,271.45947,0.0,0.0,0.0
2,2016-01-01,90.0,-175.0,-0.000055,61.945680,-10.136597,-1.500095,-12.661011,5.333832,-166.73407,271.45947,0.0,0.0,0.0
3,2016-01-01,90.0,-172.5,-0.000055,61.326180,-10.576843,-1.324284,-12.041168,5.394989,-166.61555,271.45947,0.0,0.0,0.0
4,2016-01-01,90.0,-170.0,-0.000056,60.712692,-10.988846,-1.136375,-11.409485,5.451176,-166.57408,271.45947,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26879179,2022-12-31,-90.0,167.5,0.000016,97.887580,-1.422317,2.364288,-3.507492,0.720871,-182.42667,0.00000,0.0,0.0,0.0
26879180,2022-12-31,-90.0,170.0,0.000016,97.870000,-1.215286,2.370148,-3.650070,0.809738,-182.44444,0.00000,0.0,0.0,0.0
26879181,2022-12-31,-90.0,172.5,0.000017,97.856330,-1.002396,2.372101,-3.778976,0.898605,-182.46222,0.00000,0.0,0.0,0.0
26879182,2022-12-31,-90.0,175.0,0.000017,97.844610,-0.783646,2.370148,-3.892258,0.987473,-182.58667,0.00000,0.0,0.0,0.0


In [24]:
def extract_img_from_dataset(ds):
    #ds = ds.drop(columns=['Unnamed: 0'])
    start_year = ds.time.unique()[0].split('-')[0]
    len_ds_time = len(ds.time.unique()) - 1
    end_year = ds.time.unique()[len_ds_time].split('-')[0]
    #end_year = start_year
    print(start_year,end_year)
    
    for i in range(int(start_year),int(end_year)+1):
        print(i)
        ds1 = ds.loc[(ds.time>=f"{i}-01-01") & (ds.time<f"{i+1}-01-01")]
        
        ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
        
        scaler = StandardScaler()
        cols_to_std = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']
        ds1_std = ds1
        ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])
        variables = ['time', 'latitude', 'longitude', 'vo', 'r', 'u_200', 'u_850', 'v_200', 'v_850', 'ttr', 'sst', 'shear','clim']
        ds_img_std = extract_images_new(ds1_std.loc[:,variables], 10, verbose=False)
        print('imgs are extracted')
        print(ds_img_std.shape)
        path = f'imgs/test/imgs/{i}'
        os.mkdir(path)
        np.save(f'{path}/img_{i}', ds_img_std)
        print(path)
        
        
        ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)
        y_ds1= ds1.loc[:,['new_target','latitude','longitude','time']]
        variables_y = [ 'new_target']
        y_ds1_img = extract_images_new(y_ds1, 1, verbose=False)
        print('targets are extracted')
        print(y_ds1_img.shape)
        path1 = f'imgs/test/targets/{i}'
        os.mkdir(path1)
        np.save(f'{path1}/y_{i}', y_ds1_img)
        print(path1)
        

In [25]:
extract_img_from_dataset(df11)


2016 2022
2016


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(366, 73, 144, 10)
imgs/test/imgs/2016


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(366, 73, 144, 1)
imgs/test/targets/2016
2017


...

targets are extracted
(365, 73, 144, 1)
imgs/test/targets/2021
2022


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(365, 73, 144, 10)
imgs/test/imgs/2022


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1648077009.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(365, 73, 144, 1)
imgs/test/targets/2022


In [35]:
def extract_img_from_dataset_val(ds):
    #ds = ds.drop(columns=['Unnamed: 0'])
    start_year = ds.time.unique()[0].split('-')[0]
    len_ds_time = len(ds.time.unique()) - 1
    end_year = ds.time.unique()[len_ds_time].split('-')[0]
    #end_year = start_year
    print(start_year,end_year)
    
    for i in range(int(start_year),int(end_year)+1):
        print(i)
        ds1 = ds.loc[(ds.time>=f"{i}-01-01") & (ds.time<f"{i+1}-01-01")]
        
        ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
        
        scaler = StandardScaler()
        cols_to_std = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']
        ds1_std = ds1
        ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])
        variables = ['time', 'latitude', 'longitude', 'vo', 'r', 'u_200', 'u_850', 'v_200', 'v_850', 'ttr', 'sst', 'shear','clim']
        ds_img_std = extract_images_new(ds1_std.loc[:,variables], 10, verbose=False)
        print('imgs are extracted')
        print(ds_img_std.shape)
        path = f'imgs/val/imgs/{i}'
        os.mkdir(path)
        np.save(f'{path}/img_{i}', ds_img_std)
        print(path)
        
        
        ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)
        y_ds1= ds1.loc[:,['new_target','latitude','longitude','time']]
        variables_y = [ 'new_target']
        y_ds1_img = extract_images_new(y_ds1, 1, verbose=False)
        print('targets are extracted')
        print(y_ds1_img.shape)
        path1 = f'imgs/val/targets/{i}'
        os.mkdir(path1)
        np.save(f'{path1}/y_{i}', y_ds1_img)
        print(path1)
        

In [27]:
val = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Earth/9vars/valid_earth_9var.csv')
val

,Unnamed: 0,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,0,2011-01-01,90.0,-180.0,-0.000003,99.821815,-12.349457,-3.438778,6.329109,-4.068653,-158.32777,271.4602,0.0,0.0,0.0
1,1,2011-01-01,90.0,-177.5,-0.000004,99.783510,-12.039803,-3.631245,6.906891,-3.882557,-158.36333,271.4602,0.0,0.0,0.0
2,2,2011-01-01,90.0,-175.0,-0.000004,99.707070,-11.703491,-3.815933,7.473854,-3.687809,-158.44926,271.4602,0.0,0.0,0.0
3,3,2011-01-01,90.0,-172.5,-0.000005,99.597500,-11.340897,-3.991871,8.028046,-3.485809,-158.51445,271.4602,0.0,0.0,0.0
4,4,2011-01-01,90.0,-170.0,-0.000005,99.461600,-10.952614,-4.158180,8.569008,-3.276340,-158.54110,271.4602,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19194907,3836875,2015-12-31,-90.0,167.5,0.000019,96.499310,3.515015,4.204083,-2.778534,-1.154083,-177.55481,0.0000,0.0,0.0,0.0
19194908,3836876,2015-12-31,-90.0,170.0,0.000019,96.325320,3.654198,4.288986,-2.669289,-0.958054,-177.56667,0.0000,0.0,0.0,0.0
19194909,3836877,2015-12-31,-90.0,172.5,0.000018,96.129380,3.788071,4.361580,-2.552383,-0.761833,-177.58444,0.0000,0.0,0.0,0.0
19194910,3836878,2015-12-31,-90.0,175.0,0.000018,95.915790,3.916069,4.421269,-2.427567,-0.566238,-177.70890,0.0000,0.0,0.0,0.0


In [30]:
ds1 = val
ds1 = ds1.drop(columns=['new_target'])
ds1 = ds1.drop(columns=['Unnamed: 0'])
ds1.rename(columns={"Real_tom_target": "new_target"}, inplace=True)
ds1

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target
0,2011-01-01,90.0,-180.0,-0.000003,99.821815,-12.349457,-3.438778,6.329109,-4.068653,-158.32777,271.4602,0.0,0.0
1,2011-01-01,90.0,-177.5,-0.000004,99.783510,-12.039803,-3.631245,6.906891,-3.882557,-158.36333,271.4602,0.0,0.0
2,2011-01-01,90.0,-175.0,-0.000004,99.707070,-11.703491,-3.815933,7.473854,-3.687809,-158.44926,271.4602,0.0,0.0
3,2011-01-01,90.0,-172.5,-0.000005,99.597500,-11.340897,-3.991871,8.028046,-3.485809,-158.51445,271.4602,0.0,0.0
4,2011-01-01,90.0,-170.0,-0.000005,99.461600,-10.952614,-4.158180,8.569008,-3.276340,-158.54110,271.4602,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19194907,2015-12-31,-90.0,167.5,0.000019,96.499310,3.515015,4.204083,-2.778534,-1.154083,-177.55481,0.0000,0.0,0.0
19194908,2015-12-31,-90.0,170.0,0.000019,96.325320,3.654198,4.288986,-2.669289,-0.958054,-177.56667,0.0000,0.0,0.0
19194909,2015-12-31,-90.0,172.5,0.000018,96.129380,3.788071,4.361580,-2.552383,-0.761833,-177.58444,0.0000,0.0,0.0
19194910,2015-12-31,-90.0,175.0,0.000018,95.915790,3.916069,4.421269,-2.427567,-0.566238,-177.70890,0.0000,0.0,0.0


In [6]:
df11val = tom_generate(ds1)
df11val

NameError: name 'ds1' is not defined

In [37]:
extract_img_from_dataset_val(df11val)


2011 2015
2011


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(365, 73, 144, 10)
imgs/val/imgs/2011


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(365, 73, 144, 1)
imgs/val/targets/2011
2012


...

targets are extracted
(365, 73, 144, 1)
imgs/val/targets/2014
2015


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(365, 73, 144, 10)
imgs/val/imgs/2015


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_1113/1971746298.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(365, 73, 144, 1)
imgs/val/targets/2015


In [2]:
train = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Earth/9vars/training_earth_9var.csv')


In [17]:
years=[1980,1985,1990,1995,2000,2005]
for i in years:
    tr1 = train.loc[(train.time>=f"{i}-01-01") & (train.time<f"{i+5}-01-01")]
    tr1.to_csv(f'/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/{i}_{i+5}')
    print(i)
    del tr1


1980
1985
1990
1995
2000
2005


In [6]:
np.save('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/1.csv', tr1)

In [ ]:
np.save('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/', tr1)

In [8]:
years=[1980,1985,1990,1995,2000, 2005]
for i in years:
    if i <2005:
        print(i,i+5)
    if i >= 2005:
        print(i,i+6)

1980 1985
1985 1990
1990 1995
1995 2000
2000 2005
2005 2011


In [7]:
files_name = os.listdir('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/')
files_name.remove('.DS_Store' )
files_name.remove('.npy' )

files_name


['1990_1995.csv',
 '2005_2010.csv',
 '1995_2000.csv',
 '1980_1985.csv',
 '1985_1990.csv',
 '2000_2005.csv']

In [8]:
path_folder = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/'
for i in files_name:
    print(path_folder+i)
    ds2 = pd.read_csv(path_folder+i)
    #ds2 = train
    ds2 = ds2.drop(columns=['new_target'])
    ds2 = ds2.drop(columns=['Unnamed: 0.1'])
    ds2 = ds2.drop(columns=['Unnamed: 0'])
    ds2.rename(columns={"Real_tom_target": "new_target"}, inplace=True)
    df11trn = tom_generate(ds2)
    extract_img_from_dataset_trn(df11trn)
    del ds2
    del df11trn
    


/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/1990_1995.csv
======== Done ========
1990 1994
1990


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(365, 73, 144, 10)
imgs/train/imgs/1990


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(365, 73, 144, 1)
imgs/train/targets/1990
1991


...

targets are extracted
(365, 73, 144, 1)
imgs/train/targets/2003
2004


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(366, 73, 144, 10)
imgs/train/imgs/2004


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14720/2602284585.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)


targets are extracted
(366, 73, 144, 1)
imgs/train/targets/2004


In [18]:
ds2 =  pd.read_csv('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/1980_1985.csv')
ds2 

,Unnamed: 0.1,Unnamed: 0,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,0,0,1980-01-01,90.0,-180.0,-0.000007,47.091390,10.406403,10.991116,4.897507,-0.683845,-171.17334,271.45972,0.0,0.0,0.0
1,1,1,1980-01-01,90.0,-177.5,-0.000007,47.124294,10.544151,10.880585,4.430099,-1.140003,-171.13481,271.45972,0.0,0.0,0.0
2,2,2,1980-01-01,90.0,-175.0,-0.000006,47.184170,10.662994,10.752586,3.961662,-1.587284,-171.01926,271.45972,0.0,0.0,0.0
3,3,3,1980-01-01,90.0,-172.5,-0.000007,47.268673,10.763901,10.608711,3.492065,-2.024754,-170.94221,271.45972,0.0,0.0,0.0
4,4,4,1980-01-01,90.0,-170.0,-0.000007,47.374890,10.847244,10.449337,3.022316,-2.453190,-170.92741,271.45972,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19205419,19205419,3847387,1984-12-31,-90.0,167.5,0.000020,99.802700,-1.426132,0.959446,0.882835,3.481300,-186.84056,0.00000,0.0,0.0,0.0
19205420,19205420,3847388,1984-12-31,-90.0,170.0,0.000019,99.812230,-1.447449,0.841198,0.774300,3.527267,-186.82870,0.00000,0.0,0.0,0.0
19205421,19205421,3847389,1984-12-31,-90.0,172.5,0.000018,99.816826,-1.464020,0.725212,0.665993,3.560551,-186.80500,0.00000,0.0,0.0,0.0
19205422,19205422,3847390,1984-12-31,-90.0,175.0,0.000017,99.815420,-1.474075,0.613590,0.558617,3.582737,-186.78722,0.00000,0.0,0.0,0.0


In [14]:
ds2

array([[0, '1980-01-01', 90.0, ..., 0.0, 0.0, 0.0],
       [1, '1980-01-01', 90.0, ..., 0.0, 0.0, 0.0],
       [2, '1980-01-01', 90.0, ..., 0.0, 0.0, 0.0],
       ...,
       [3847389, '1984-12-31', -90.0, ..., 0.0, 0.0, 0.0],
       [3847390, '1984-12-31', -90.0, ..., 0.0, 0.0, 0.0],
       [3847391, '1984-12-31', -90.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [7]:
df11trn = tom_generate(ds2)
df11trn

======== Done ========


,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,1980-01-01,90.0,-180.0,-0.000007,47.091390,10.406403,10.991116,4.897507,-0.683845,-171.17334,271.45972,0.0,0.0,0.0
1,1980-01-01,90.0,-177.5,-0.000007,47.124294,10.544151,10.880585,4.430099,-1.140003,-171.13481,271.45972,0.0,0.0,0.0
2,1980-01-01,90.0,-175.0,-0.000006,47.184170,10.662994,10.752586,3.961662,-1.587284,-171.01926,271.45972,0.0,0.0,0.0
3,1980-01-01,90.0,-172.5,-0.000007,47.268673,10.763901,10.608711,3.492065,-2.024754,-170.94221,271.45972,0.0,0.0,0.0
4,1980-01-01,90.0,-170.0,-0.000007,47.374890,10.847244,10.449337,3.022316,-2.453190,-170.92741,271.45972,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119027371,2010-12-31,-90.0,167.5,0.000020,100.105100,6.575180,3.155399,-2.889877,0.126186,-179.84277,0.00000,0.0,0.0,0.0
119027372,2010-12-31,-90.0,170.0,0.000020,100.081600,6.694992,3.171963,-2.677902,0.297466,-179.86649,0.00000,0.0,0.0,0.0
119027373,2010-12-31,-90.0,172.5,0.000020,100.046690,6.808556,3.180149,-2.460533,0.465012,-179.89612,0.00000,0.0,0.0,0.0
119027374,2010-12-31,-90.0,175.0,0.000020,100.001850,6.914993,3.181087,-2.237625,0.628326,-180.05019,0.00000,0.0,0.0,0.0


In [ ]:
np.save('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Earth/9var ok/', df11trn)

In [12]:
def extract_img_from_dataset_trn(ds):
    #ds = ds.drop(columns=['Unnamed: 0'])
    start_year = ds.time.unique()[0].split('-')[0]
    len_ds_time = len(ds.time.unique()) - 1
    end_year = ds.time.unique()[len_ds_time].split('-')[0]
    #end_year = start_year
    print(start_year,end_year)
    
    for i in range(int(start_year),int(end_year)+1):
        print(i)
        ds1 = ds.loc[(ds.time>=f"{i}-01-01") & (ds.time<f"{i+1}-01-01")]
        
        ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
        
        scaler = StandardScaler()
        cols_to_std = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']
        ds1_std = ds1
        ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])
        variables = ['time', 'latitude', 'longitude', 'vo', 'r', 'u_200', 'u_850', 'v_200', 'v_850', 'ttr', 'sst', 'shear','clim']
        ds_img_std = extract_images_new(ds1_std.loc[:,variables], 10, verbose=False)
        print('imgs are extracted')
        print(ds_img_std.shape)
        path = f'imgs/train/imgs/{i}'
        os.mkdir(path)
        np.save(f'{path}/img_{i}', ds_img_std)
        print(path)
        
        
        ds1['new_target'] = ds1.apply(lambda x: 1 if x.new_target>=50 else 0,axis=1)
        y_ds1= ds1.loc[:,['new_target','latitude','longitude','time']]
        variables_y = [ 'new_target']
        y_ds1_img = extract_images_new(y_ds1, 1, verbose=False)
        print('targets are extracted')
        print(y_ds1_img.shape)
        path1 = f'imgs/train/targets/{i}'
        os.mkdir(path1)
        np.save(f'{path1}/y_{i}', y_ds1_img)
        print(path1)
        

In [ ]:
extract_img_from_dataset_trn(df11trn)

1980 2010
1980


In [6]:
path_folder = '/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/'
i = '1990_1995.csv'
print(path_folder+i)
ds2 = pd.read_csv(path_folder+i)
    #ds2 = train
ds2 = ds2.drop(columns=['new_target'])
ds2 = ds2.drop(columns=['Unnamed: 0.1'])
ds2 = ds2.drop(columns=['Unnamed: 0'])
ds2.rename(columns={"Real_tom_target": "new_target"}, inplace=True)
df11trn = tom_generate(ds2)
extract_img_from_dataset_trn(df11trn)
del ds2
del df11trn

/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/data_training/1990_1995.csv
======== Done ========
1990 1994
1990


/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14294/2602284585.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1['shear'] = ds1.apply(lambda x: sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
/var/folders/6l/2k5dphhj4z30f5zpqx44561c0000gn/T/ipykernel_14294/2602284585.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds1_std[cols_to_std] = scaler.fit_transform(ds1[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst','shear','clim']])


imgs are extracted
(365, 73, 144, 10)
imgs/train/imgs/1990


KeyboardInterrupt: 

In [13]:
train = pd.read_csv('/Users/3rfanian/P_main/extracting data 10 variable/dataframes/Earth/9vars/training_earth_9var.csv')


In [14]:
i = 2010
tr1 = train.loc[(train.time>=f"{i}-01-01") & (train.time<f"{i+1}-01-01")]
tr1

,Unnamed: 0,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
115190496,0,2010-01-01,90.0,-180.0,0.000021,92.556550,-7.869583,0.792339,-10.983681,-15.071434,-140.93427,271.45972,0.0,0.0,0.0
115190497,1,2010-01-01,90.0,-177.5,0.000016,92.644585,-8.363083,0.041466,-10.626030,-15.056622,-140.95204,271.45972,0.0,0.0,0.0
115190498,2,2010-01-01,90.0,-175.0,0.000012,92.723710,-8.837021,-0.709156,-10.238968,-15.020618,-141.18315,271.45972,0.0,0.0,0.0
115190499,3,2010-01-01,90.0,-172.5,0.000007,92.796740,-9.290512,-1.459187,-9.822685,-14.962685,-141.34315,271.45972,0.0,0.0,0.0
115190500,4,2010-01-01,90.0,-170.0,0.000003,92.864890,-9.720078,-2.207622,-9.378906,-14.882683,-141.39648,271.45972,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119027371,3836875,2010-12-31,-90.0,167.5,0.000020,100.105100,6.575180,3.155399,-2.889877,0.126186,-179.84277,0.00000,0.0,0.0,0.0
119027372,3836876,2010-12-31,-90.0,170.0,0.000020,100.081600,6.694992,3.171963,-2.677902,0.297466,-179.86649,0.00000,0.0,0.0,0.0
119027373,3836877,2010-12-31,-90.0,172.5,0.000020,100.046690,6.808556,3.180149,-2.460533,0.465012,-179.89612,0.00000,0.0,0.0,0.0
119027374,3836878,2010-12-31,-90.0,175.0,0.000020,100.001850,6.914993,3.181087,-2.237625,0.628326,-180.05019,0.00000,0.0,0.0,0.0


In [16]:
#del train
ds2 = tr1
ds2 = ds2.drop(columns=['new_target'])
#ds2 = ds2.drop(columns=['Unnamed: 0.1'])
ds2 = ds2.drop(columns=['Unnamed: 0'])
ds2.rename(columns={"Real_tom_target": "new_target"}, inplace=True)
df11trn = tom_generate(ds2)
extract_img_from_dataset_trn(df11trn)

======== Done ========
2010 2010
2010
imgs are extracted
(365, 73, 144, 10)
imgs/train/imgs/2010
targets are extracted
(365, 73, 144, 1)
imgs/train/targets/2010


In [3]:
img_2011 = np.load('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/val/imgs/2011/img_2011.npy')
y_2011 = np.load('/Users/3rfanian/P_main/TC-main/Paolo/Notebooks_newTarget/imgs/val/targets/2011/y_2011.npy')


In [7]:
print(img_2011.shape)
img_2011

(365, 73, 144, 10)


array([[[[-1.39946294e-01,  1.64429915e+00, -1.48921297e+00, ...,
           5.95798593e-01, -3.84822956e-01, -2.13051244e-01],
         [-1.57765907e-01,  1.64287669e+00, -1.47145571e+00, ...,
           5.95798593e-01, -3.85974479e-01, -2.13051244e-01],
         [-1.76794521e-01,  1.64003810e+00, -1.45216973e+00, ...,
           5.95798593e-01, -3.86901395e-01, -2.13051244e-01],
         ...,
         [-9.88094602e-02,  1.63902268e+00, -1.53291729e+00, ...,
           5.95798593e-01, -3.80437692e-01, -2.13051244e-01],
         [-1.09961739e-01,  1.64239245e+00, -1.51996257e+00, ...,
           5.95798593e-01, -3.81994028e-01, -2.13051244e-01],
         [-1.23765250e-01,  1.64416008e+00, -1.50540613e+00, ...,
           5.95798593e-01, -3.83451646e-01, -2.13051244e-01]],

        [[-4.05528186e-01,  1.60277451e+00, -1.39287644e+00, ...,
           5.95798593e-01, -4.01413262e-01, -2.13051244e-01],
         [-5.02902399e-01,  1.61048922e+00, -1.37327942e+00, ...,
           5.95798593e

In [8]:
print(y_2011.shape)

y_2011

(365, 73, 144, 1)


array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 